# Chapter 3 - Optimal Flows (Julia Code)

## Bellman’s Method

Our first step is to set up the cost function, which we store as an array called $c$. Note that we set $c[i, j] = Inf$ when no edge
exists from $i$ to $j$, so that such a path is never chosen when evaluating the Bellman operator

In [1]:
c = fill(Inf, (7, 7))
c[1, 2], c[1, 3], c[1, 4] = 1, 5, 3
c[2, 4], c[2, 5] = 9, 6
c[3, 6] = 2
c[4, 6] = 4
c[5, 7] = 4
c[6, 7] = 1
c[7, 7] = 0

0

Next we define the Bellman operator.

In [2]:
function T(q)
    Tq = similar(q)
    n = length(q)
    for x in 1:n
        Tq[x] = minimum(c[x, :] + q[:])
    end
    return Tq
end

T (generic function with 1 method)

Now we arbitraryly set $𝑞 ≡ 0$, generate the sequence of iterates $𝑇𝑞$, $𝑇^2𝑞$, $𝑇^3𝑞$ and plot them.

In [3]:
using PyPlot
fig, ax = plt.subplots()

n = 7
q = zeros(n)
ax.plot(1:n, q)

for i in 1:3
    new_q = T(q)
    ax.plot(1:n, new_q, "-o", alpha=0.7, label ="iterate $i")
    q = new_q
end

ax.legend()

LoadError: InitError: PyError (PyImport_ImportModule

The Python package matplotlib could not be imported by pyimport. Usually this means
that you did not install matplotlib in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/usr/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the matplotlib module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build(

## Linear programming
When solving linear programs, one option is to use a domain specific modeling language to set out the objective and constraints in the optimization problem. Here we demonstrate the Julia package JuMP.

In [ ]:
using JuMP
using GLPK

We create our model object and select our solver.

In [ ]:
m = Model()
set_optimizer(m, GLPK.Optimizer)


Now we add variables, constraints and an objective to our model.

In [ ]:
@variable(m, q1 >= 0)
@variable(m, q2 >= 0)
@constraint(m, 2q1 + 5q2 <= 30)
@constraint(m, 4q1 + 2q2 <= 20)
@objective(m, Max, 3q1 + 4q2)

Finally we solve our linear program.

In [ ]:
optimize!(m)

println(value.(q1)) 
println(value.(q2))